In [ ]:
# Импорт необходимых модулей 
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = 'black'
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Работа с текстом

До этого мы работали с численными данными или категориями, которые были представлены строками. Строки мы научились кодировать, но что если данные - это не просто строка или число, а осмысленный текст (заявка пользователей, письмо, комментарий на сайте, отзыв). Что нам тогда делать?

Практики по тексту мы делать не будем, а просто познакомимся с новыми фреймворками и подходами прямо здесь! Нашей сегодняшней задачей будет классификация новостей по категориям, для этого воспользуемся вот [этим набором данных](https://scikit-learn.org/stable/datasets/index.html#newsgroups-dataset).

В чем сложность работы с текстом? В первую очередь, это то, что текст - это вообще не числа, а модели, как и раньше, могут работать только с числами. Поэтому нашей задачей будет преобразовать текст в численное представление. Таким образом мы понакомимся с одним из способов кодирования слов и предложений в числа (а точнее, числовые вектора).

Начнем с того, что посмотрим на наши данные:

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# В функции загрузки уже есть разделение на обучение/тест
#   воспользуемся этим на момент подготовки модели
# Для анализа лучше посмотреть на все данные
newsgroups_data = fetch_20newsgroups(subset='all', random_state=RANDOM_STATE)

In [ ]:
print(newsgroups_data.keys())

In [ ]:
# Посмотрим, какой у данных тип
data = newsgroups_data['data']
targets = newsgroups_data['target']
target_names = newsgroups_data['target_names']

print(f"Data type:\t{type(data)}\n")
print(f"Target names:\n{target_names}\n")
print(f"Target data:\n{targets[:10]}")

Таакс, данные - это какой-то список и мы скоро узнаем какой именно, целевые переменные - индексы категорий. Посмотрим на пример данных:

In [ ]:
print(data[0])
print('--------------')
print(f'Target: {target_names[targets[0]]}')

Как видим, данные представляют собой список текстов, поэтому предобработка будет связана с анализом и обработкой текстовой информациий.

# BOW

Так, данные мы видим, но только тут две проблемы:
- Модель работает с числами, а у нас куча слов;
- В тексте может быть 100 слов, а может быть 1000, как нам подавать неопределенное количество слов на вход модели?

Самым простым подходом является метод **Bag Of Words**. Суть его заключается в том, что мы составляем список всех возможных слов из всех текстов (**словарь**). После этого мы можем сформировать числовую матрицу, в которой строки - тексты набора данных, а колонки - слова из словаря.

Сам подход, когда мы преобразуем что-то в числовой вектор фиксированной длины называется **векторизация**. То есть мы делаем преобразование данных неопределенной длины в вектор фиксированной длины, с которым модель уже может сама работать.

Посмотрим:

In [ ]:
# Здесь специально сделана некоторая предобработка, 
#   которая обычно делается в рамках подготовки
texts_dataset = [
    "пирожок это лишь пирожок",
    "пирог не кушать пирожок можно",
    "сегодня ходил кино поел пирог"
]

corpus = set()
# Для начала составим словарь
for text in texts_dataset:
    tokens = text.split(' ')
    corpus.update(tokens)

corpus = list(corpus)
print(f'Corpus: {corpus}')

In [ ]:
# После составления корпуса мы можем составить матрицу попаданий
samples_count = len(texts_dataset)
corpus_len = len(corpus)
X_data = np.zeros((samples_count, corpus_len), dtype=int)

for i_sample, text in enumerate(texts_dataset):
    tokens = text.split(' ')
    for token in tokens:
        token_index = corpus.index(token)
        X_data[i_sample, token_index] += 1

# Для лучшего представления составим DataFrame
X_df = pd.DataFrame(X_data, columns=corpus)
X_df['_texts'] = texts_dataset

X_df

Отлично! Теперь вне зависимости от длины текста мы для каждого текста имеем закодированное представление в виде числового вектора! То что надо для модели!

Только есть два момента:
- Векторы не нормированы;
- Теряется порядок слов.

Как мы можем отнормировать векторы? Да просто поделить на количество слов в каждом тексте, то есть для каждой строки находим сумму элементов в строке и всю строку делим на нее:

In [ ]:
row_sums = X_data.sum(axis=1)
X_data_norm = X_data/row_sums[:,None]

X_df = pd.DataFrame(X_data_norm, columns=corpus)
X_df['_texts'] = texts_dataset

X_df

Мы получили в итоге еще и нормированные вектора для каждого текста.

Что же делать с порядком слов? Мы можем вместо разовых слов проверять еще и комбинации слов, например *good movie*, *did not* и другие. Комбинации из двух слов называются биграммы, из трех - триграммы и так общее название - n-граммы (n-grams). Это позволяет учитывать порядок в словах, а также еще и очень сильно расширяет словарь!

Обратите внимание, что с введением нормирования векторы теперь сильнее отличаются друг от друга, нежели были раньше, но даже тут почти весь вектор состоит из одних и тех же значений. При больших объемах вектора почти не будут отличаться друг от друга - как тогда модели их разделять?

По сути подход BOW - отличный вариант, когда тексты небольшие и корпусы (словари) получаются маленькими. Основной проблемой этого подхода является количество признаков в матрице (то есть размер словаря), которые летят на вход модели, и похожесть векторов между собой. Давайте рассмотрим другой подход, чуть сложнее, но он стоит того!

# TF-IDF


Название **TF-IDF** формируется из следующих названий: 
- TF (term frequency) - отношение числа вхождений слова в тексте к числу слов в тексте;
- IDF (inverse document frequency) - логарифм количества текстов к количеству текстов с этим словом. 

Результирующий показатель формируется из умножения значений TF на IDF для каждого интересующего слова. То есть, показатель $TFIDF = TF * IDF$.

Давайте попробуем разобраться в подходе! Наша задача - вычислить два показателя (TF и IDF) и после этого умножить их.

По сути мы уже создавали вектора в формате TF, когда для каждого слова рассчитывали сколько раз он попадается в каждом тексте к общему числу слов в этом тексте. Можно сказать, что нормированный BOW - это TF!

Давайте для примера рассчитаем TFIDF для слова пирожок в наборе текстов:
- "Пирожок - это лишь пирожок!"
- "Пирог не кушать, пирожок - можно."
- "Я сегодня ходил в кино и поел пирог!"

$$
TF(word) = W(word) / N;
$$
где $W$ - количество упоминаний слова в тексте, $N$ - количество слов в тексте.

Для нашего случая получается по каждому тексту (нижний индекс):
$$
TF_1(пирожок) = 2 / 4 = 0.5; \\
TF_2(пирожок) = 1 / 5 = 0.2; \\
TF_3(пирожок) = 0 / 8 = 0; \\
$$

Теперь остается понять, как рассчитывать IDF. По сути, обратная частота документов - это сколько мы имеем текстов со словом по отношению к количеству текстов, только наоборот! Он показывает, в скольки текстах упоминается это слово.

$$
IDF(word) = log(M / T(word));
$$
где $T$ - количество текстов с этим словом в наборе данных, $M$ - количество текстов в наборе данных.

А значит для нас:
$$
IDF(пирожок) = log(3/2) = 0.176;
$$

Итого:
$$
TFIDF_1(пирожок) = 0.5 * 0.176 = 0.088; \\
TFIDF_2(пирожок) = 0.2 * 0.176 = 0.035; \\
TFIDF_3(пирожок) = 0 * 0.176 = 0; \\
$$

Вот такая математика! Но для чего это нужно? У нас же уже есть отличный BOW с нормированием. IDF на самом деле делает очень важную вещь - помимо того, что мы оцениваем с помощью TF, насколько "важно" слово в отдельных текстах, с помощью IDF мы еще и устраиваем нормирование с уточнением, насколько слово важно во всех текстах, так сказать, тематическое нормирование. Но не нужно сильно ориентироваться на "важно", тут скорее играет большую роль тот факт, что каждое слово имеет дополнительную характеристику, что сделает вектор более уникальным.

Давайте посмотрим на примере, как работает TFIDF в рамках инструмента из фреймворка `sklearn` - [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    # Ограничение максимального кол-ва признаков (размера выходного вектора)
    #   None -> не ограничено, вычисляется из данных
    max_features=None,
)

X_data = [
    "Пирожок - это лишь пирожок!",
    "Пирог не кушать, пирожок - можно.",
    "Я сегодня ходил в кино и поел пирог!"
]

X_data_vec = vectorizer.fit_transform(X_data)
# Отобразим векторизированное представление (кол-во данных, кол-во фич)
print(X_data_vec.shape)

In [ ]:
# Мы можем проверить корпус, который сформировался при генерации
corpus = vectorizer.get_feature_names()
corpus

In [ ]:
dict(zip(corpus, vectorizer.idf_))

In [ ]:
df = pd.DataFrame(X_data_vec.todense(), columns = corpus)
df['_texts'] = X_data

df

Можно заметить, что численные значения отличаются от наших расчетов. Все верно, так как внутренняя формула для расчета немного отличается от той, что мы рассмотрели. Но не так важно получить конкретные числа (до тех пор, пока мы не хотим реализовать свой TFIDF трансформер), как понять, что каждый текст представлен в виде численного вектора, который может быть подан на вход модели!

> Еще обратите внимание, что трансформер сам делает некоторую предобработку и слова, в которых всего одна буква - исключены из корпуса.

В результате из-за того, что помимо частоты слова в конкретном тексте, мы оцениваем еще и насколько часто слово встречается среди текстов, векторы отличаются сильнее, что нам и нужно! По сути нам не так важны значения векторов, нежели их различия в значениях между собой, чтобы модель понимала отличия и вытаскивала информацию из этого!

# Байесовский классификатор

Мы уже знаем различные модели, но специфика данных в данном случае: они кодируются с частотой появления в текстах и, как результат, матрица данных имеет очень много фич (столбцов). Не нужно говорить, что для деревьев такой случай не очень подходит из-за количества фич, а линейная регрессия может оказаться недостаточной, поэтому мы познакомимся с подходом, который основывается на теореме Байеса и хорошо работает с закодированными данными в виде частот появлений.

Напомним, что теорема выглядит так:
$$
P(A|B) = \frac{P(B|A)P(A)}{P(B)}
$$

Здесь расшифровка следующая:
$P(A|B)$ - вероятность $A$ при истинности $B$;
$P(B|A)$ - вероятность $B$ при истинности $A$;
$P(A)$ - априорная вероятность того, что $A$ истинно;
$P(B)$ - априорная вероятность того, что $B$ истинно.

Сложно? На сегодняшний день очень много статей в интернете, объясняющих теорему, а мы будем сразу применять нашу теорему к данным и классификации!

Для примера возьмем следующие данные:

In [ ]:
df = pd.DataFrame({
    'x1': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2],
    'x2': ['A', 'B', 'B', 'B', 'C', 'A', 'A', 'C', 'A', 'A', 'C', 'B', 'B', 'C', 'C'],
    'y': [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0]
})

df

Давайте попробуем с помощью байесовской теоремы определить, к какому классу принадлежит новая запись $X_{new} = [2, C]$ данных?

По сути нашей задачей является определение вероятностей $P(y=1|X=X_{new})$ и $P(y=0|X=X_{new})$. Первая вероятность - это вероятность того, что новая запись принадлежит классу 1 (то есть вероятность выбора класса 1 при условии появления данных $X_{new}$). Вторая вероятность - это вероятность классифицировать новую запись как класс 0 (аналогично). Какая вероятность будет больше - тот класс и выберем!

Теперь распишем формулы:
$$
P(y=1|X=X_{new}) = \frac{P(X=X_{new}|y=1)P(y=1)}{P(X=X_{new})} \\
P(y=0|X=X_{new}) = \frac{P(X=X_{new}|y=0)P(y=0)}{P(X=X_{new})} \\
$$

По сути, из-за того, что знаменатели одинаковые, то нам нужно всего-то сравнить числители! Не будем усложнять себе жизнь, надо определить всего-то 4 показателя:
- $P(y=0)$;
- $P(y=1)$;
- $P(X=X_{new}|y=0)$;
- $P(X=X_{new}|y=1)$.

Начнем с первых двух, это априорные вероятности, то есть, нам просто нужно определить, насколько вероятно на основе данных получить запись с классом $y=0$ или $y=1$. Как это сделать? Поделить количество записей класса на общее количество записей!

In [ ]:
df['y'].value_counts()

Вот так мы получили количество записей классов, а значит может рассчитать априорные вероятности:
$$
P(y=0) = \frac{7}{15}, P(y=1) = \frac{8}{15}
$$

Не сложно, не так ли? А что нам делать с остальными двумя? Тут немного сложнее, нам нужно рассчитать вероятности получения определенных значений фич при условии, что выбран конкретный класс. То есть, чтобы посчитать $P(X=X_{new}|y=1)$ для новой записи $X_{new} = [2, C]$, нам нужно раскрыть это выражение в виде $P(x_1=2|y=1)*P(x_2=C|y=1)$. Аналогично для другого класса. Итого, мы получаем:
$$
P(X=X_{new}|y=1) = P(x_1=2|y=1)*P(x_2=C|y=1) \\
P(X=X_{new}|y=0) = P(x_1=2|y=0)*P(x_2=C|y=0) \\
$$

> Тут по сути раскрытие $P(X) \rightarrow P(x_1)*P(x_2)$, так как вектор X состоит из двух компонент.

Давайте посмотрим, какие показатели нам нужны для вероятностей:

In [ ]:
df.loc[df['y'] == 1, 'x1'].value_counts()

Таакс, мы можем рассчитать нашу первую вероятность, по сути поделив количество записей, в которых $x_1$ равен 2 и эти записи принадлежат классу $y = 1$ на общее количество записей этого класса:
$$
P(x_1=2|y=1) = \frac{3}{8}
$$

По аналогии считаем остальные показатели:

In [ ]:
df.loc[df['y'] == 1, 'x2'].value_counts()

In [ ]:
df.loc[df['y'] == 0, 'x1'].value_counts()

In [ ]:
df.loc[df['y'] == 0, 'x2'].value_counts()

Итого получаем:
$$
P(x_1=2|y=1) = \frac{3}{8}, 
P(x_2=С|y=1) = \frac{2}{8} \\
P(x_1=2|y=0) = \frac{2}{7}, 
P(x_2=С|y=0) = \frac{3}{7} \\
$$

И считаем наши вероятности:
$$
P(y=1|X=X_{new}) = P(X=X_{new}|y=1)P(y=1) = \frac{3}{8} * \frac{2}{8} * \frac{8}{15} = \frac{1}{20} \\
P(y=0|X=X_{new}) = P(X=X_{new}|y=0)P(y=0) = \frac{2}{7} * \frac{3}{7} * \frac{7}{15} = \frac{2}{35}
$$

Так как $\frac{1}{20} < \frac{2}{35}$, то вывод - новая запись $X_{new} = [2, C]$ будет присвоена классу $y=0$!

Обратите внимание, классификатору для предсказания не потребовалось ничего, кроме данных и пары расчетов вероятностей. Именно поэтому данные модели хорошо справляются в задачами, в которых данные представлены в виде частот или количества повторений!

Интерфейс работы с ничем не отличается от того, что мы привыкли, вот класс [MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html):

In [ ]:
# Подгрузим данные
newsgroups_train = fetch_20newsgroups(
    subset='train', random_state=RANDOM_STATE)
newsgroups_test = fetch_20newsgroups(
    subset='test', random_state=RANDOM_STATE)

X_train = newsgroups_train['data']
y_train = newsgroups_train['target']

X_test = newsgroups_test['data']
y_test = newsgroups_test['target']

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Без всяких предобработок кидаем, что есть в трансформацию
vectorizer = TfidfVectorizer()

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

nb_clf = MultinomialNB()
nb_clf.fit(X_train_vec, y_train)

print(f'Train accuracy: {nb_clf.score(X_train_vec, y_train)}')
print(f'Test accuracy: {nb_clf.score(X_test_vec, y_test)}')

Вот так мы построили какую-то модель, которая немного переобучилась, судя по метрике точности.

> Никогда не делайте выводы о результатах по единственной метрике, этого недостаточно, чтобы оценить результаты!

# Предобработка данных

Мы, конечно, увидели, что можно текст кинуть на работу модели просто прогнав через трансформацию, но работа с текстом имеет и свои подходы расширенного анализа и предобработки! Рассмотрим некоторые основные из них!

При работе с текстом наиболее распространенными практиками являются следующие способы предоработки:
- Приведение к нижнему регистру - позволяет работать с единой формой слов, при которой не разницы в словах Hello и hello;
- Удаление пунктуации, стоп-слов и низкочастотных слов - позволяет акценитровать внимание на значимых словах;
- Токенизация - разбиение предложений на токены (слова, биграммы, триграммы, N-граммы);
- Стеммизация - удаление суффиксов слов: playing -> play, studied -> studi, один из методов нормализации;
- Лемматизация - приведение к лемме (нормальной форме): played -> play, тоже один из подходов нормализации;

> Насчет паттернов (`re` ~ `RegularExpressions`) и того, как ими пользоваться есть хорошая статья: https://tproger.ru/translations/regular-expression-python/

Для реализации необходимого функционала воспользуемся фреймворком `nltk` и загрузкой необходимых данных через `nltk.download()`.

> Альтернативными фреймворками для обработки текста являются фреймворки [spaCy](https://spacy.io/), [TextBlob](https://textblob.readthedocs.io/en/dev/), [Gensim](https://radimrehurek.com/gensim/). Вы можете попробовать использовать их, так как каждый из них имеет свои плюсы и минусы.

In [ ]:
import nltk
# Скачиваем необходимые модули фреймворка nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from string import punctuation

Теперь опробуем каждую из методик:

In [ ]:
# Выберем текст для примера и посмотрим
sample_text = X_train[1]

print(sample_text)

In [ ]:
# Проверим работу приведения к нижнему регистру
# Этот подход позволяет исключить различия слов Hello и hello,
#   так как по сути это одно и то же слово
sample_text = sample_text.lower()
print(sample_text)

In [ ]:
# Удаление пунктуации
# Более сложный анализ может учитывать пунктуацию, но для простых
#   случаев пунктуация исключается, чтобы оставить лишь слова
#   как основную информацию
punct_transl = str.maketrans('', '', punctuation)
sample_text = sample_text.translate(punct_transl)
print(sample_text)

In [ ]:
# Удаление чисел
# Числа как правило редко повторяются, для простого подхода
#   достаточно удалить числа, так как это неповторяющаяся информация
sample_text = re.sub(r'\d+', '', sample_text)
print(sample_text)

In [ ]:
# Удаление повторяющихся пробелов
# Часто в текста делают кучу пробелов и отступов - они не несут информации
sample_text = re.sub(r'\s+', ' ', sample_text)
print(sample_text)

In [ ]:
# Токенизация - превращаем одну большую строку в массив токенов (слов)
# Под токенами могут пониматься не только слова, но и комбинации слов,
#   хотя для простого анализа - достаточно токенизировать до слов
word_tokens = nltk.word_tokenize(sample_text)
print(word_tokens)

In [ ]:
# Удаляем стоп-слова, для начала посмотрим, что это за слова
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:
# Теперь фильтруем стоп-слова из наших токенов
word_tokens = [word for word in word_tokens if word not in stop_words]
print(word_tokens)

In [ ]:
# Проводим лемматизацию - приводим к нормальной форме
wordnet_lemmatizer = WordNetLemmatizer()

word_tokens = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
print(word_tokens)

In [ ]:
# Чтобы лучше понять, как он работает - рассмотрим примеры:
print(wordnet_lemmatizer.lemmatize('bats'))
print(wordnet_lemmatizer.lemmatize('are'))  # Ууупс, тут не преобразовалось в be =(
print(wordnet_lemmatizer.lemmatize('feet'))

In [ ]:
# После этого нам нужно объединить токены обратно в единую строку 
#   для будущего кодирования
processed_text = ' '.join(word_tokens)
print(processed_text)

Как вы видите, каждый этап предобработки делает простую трансформацию данных, чтобы в конечном итоге привести к более стандартному виду без примесей и лишних выбросов. При этом один из последних этапов - лемматизация может иметь ошибки в работе. Аналогом лемматизации является стемминг, когда вместо приведения к нормальной форме мы отбрасывает окончание и суффикс. Выбор подхода остается за разработчиком и конкретной задачей.

# Задачи

- Проведите анализ текстовых данных набора данных 20newsgroups;
- Постройте модель `MultinominalNB` с параметрами по-умолчанию и без предобработки текста, только векторизация (baseline);
- Добавьте предобработку текста, оцените работу модели;
- Отобразите CM и сделайте выводы, какие разделы путаются;
- Изучите влияние $\alpha$ байесовского классификатора (7 значений);
- Изучите влияние максимального количества признаков `TfidfVectorizer` (10 значений `max_features`);
- Оцените работу модели при различных значения `ngram_range` (варьируя верхний предел - от 1 до 3 включительно);
- Определите лучшую модель путем настройки гиперпараметров (по показателю `f1_macro`), отобразите CM и отчет по классификации;
- Постройте модель случайного леса, определите лучшие гиперпараметры и сравните с моделью байесовского классификатора;
- Проанализируйте ошибки классификаторов и сделайте выводы;
- Удалите дополнительную информацию (используйте аргумент `remove` в функции `fetch_20newsgroups()` с указанием в качестве значения кортеж `('headers', 'footers', 'quotes')`) -> оцените работу лучшей модели (с переобучением на очищенном наборе данных);
- Определите новую лучшую модель (с учетом удаления доп.информации из текстов) путем настройки гиперпараметров (по показателю `f1_macro`);
- Отобразите CM и сделайте выводы, какие разделы путаются;

# Вопросы

- За что отвечает параметр $\alpha$ в модели? Как значение влияет на показатели 
метрик?
- В чем разница работы модели случайного леса и байесовского классификатора?
- Как повлияло удаление дополнительной информации из текстов? С чем это может быть связано?